# Deep-Emotion

In [ ]:
!pip install ipywidgets --upgrade

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt

In [ ]:
class Deep_Emotion(nn.Module):
    def __init__(self):
        super(Deep_Emotion,self).__init__()
        self.conv1 = nn.Conv2d(1,10,3)
        self.conv2 = nn.Conv2d(10,10,3)
        self.pool2 = nn.MaxPool2d(2,2)

        self.conv3 = nn.Conv2d(10,10,3)
        self.conv4 = nn.Conv2d(10,10,3)
        self.pool4 = nn.MaxPool2d(2,2)

        self.norm = nn.BatchNorm2d(10)

        self.fc1 = nn.Linear(810,50)
        self.fc2 = nn.Linear(50,7)

        self.localization = nn.Sequential(
            nn.Conv2d(1, 8, kernel_size=7),
            nn.MaxPool2d(2, stride=2),
            nn.ReLU(True),
            nn.Conv2d(8, 10, kernel_size=5),
            nn.MaxPool2d(2, stride=2),
            nn.ReLU(True)
        )

        self.fc_loc = nn.Sequential(
            nn.Linear(640, 32),
            nn.ReLU(True),
            nn.Linear(32, 3 * 2)
        )
        self.fc_loc[2].weight.data.zero_()
        self.fc_loc[2].bias.data.copy_(torch.tensor([1, 0, 0, 0, 1, 0], dtype=torch.float))

    def stn(self, x):
        xs = self.localization(x)
        xs = xs.view(-1, 640)
        theta = self.fc_loc(xs)
        theta = theta.view(-1, 2, 3)

        grid = F.affine_grid(theta, x.size())
        x = F.grid_sample(x, grid)
        return x
    
    def forward(self,input):
        out = self.stn(input)

        out = F.relu(self.conv1(out))
        out = self.conv2(out)
        out = F.relu(self.pool2(out))

        out = F.relu(self.conv3(out))
        out = self.norm(self.conv4(out))
        out = F.relu(self.pool4(out))

        out = F.dropout(out)
        out = out.view(-1, 810)
        out = F.relu(self.fc1(out))
        out = self.fc2(out)

        return out

In [ ]:
model = Deep_Emotion()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
train_dir = '/kaggle/input/fer2013/train'
test_dir = '/kaggle/input/fer2013/test'

transform = transforms.Compose([
    transforms.Grayscale(), 
    transforms.Resize((48, 48)),  
    transforms.ToTensor(),  
    transforms.Normalize(mean=[0.5], std=[0.5])  
])

train_dataset = ImageFolder(train_dir, transform=transform)
test_dataset = ImageFolder(test_dir, transform=transform)

batch_size = 64

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
def plot_metrics(train_losses, train_accs, test_losses, test_accs):
    plt.figure(figsize=(15, 5))

    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label='Train Loss')
    plt.plot(test_losses, label='Test Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training and Test Losses')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(train_accs, label='Train Accuracy')
    plt.plot(test_accs, label='Test Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.title('Training and Test Accuracies')
    plt.legend()

    plt.tight_layout() 
    plt.show()


In [ ]:
def train(model, criterion, optimizer, train_loader, device, epochs=20):
    model.train()
    model.to(device)

    train_losses = []
    train_accs = []

    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0

        for batch_idx, (data, target) in enumerate(train_loader):
            try:
                data, target = data.to(device), target.to(device)
                optimizer.zero_grad()
                output = model(data)

                loss = criterion(output, target)
                loss.backward()
                optimizer.step()

                running_loss += loss.item()
                _, predicted = output.max(1)
                total += target.size(0)
                correct += predicted.eq(target).sum().item()

                if batch_idx % 100 == 99:
                    print(f"Epoch {epoch+1}/{epochs}, "
                          f"Batch {batch_idx+1}/{len(train_loader)}, "
                          f"Loss: {running_loss/100:.4f}, "
                          f"Train Acc: {(100 * correct / total):.2f}%")
                    running_loss = 0.0
                    correct = 0
                    total = 0

            except RuntimeError as e:
                print(f"RuntimeError: {e}")
                print(f"Batch index: {batch_idx}, Data shape: {data.shape}, Target shape: {target.shape}")
                break

        epoch_loss = running_loss / len(train_loader)
        epoch_acc = 100 * correct / total

        train_losses.append(epoch_loss)
        train_accs.append(epoch_acc)

        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.2f}%")

    return train_losses, train_accs


In [ ]:
train_losses, train_accs = train(model, criterion, optimizer, train_loader, device, epochs=50)

In [ ]:
def test(model, criterion, test_loader, device, epochs=1):  # Added epochs parameter with a default value of 1
    model.eval()
    model.to(device)
    
    test_losses = []
    test_accs = []
    
    with torch.no_grad():
        for epoch in range(epochs):
            test_loss = 0
            correct = 0
            total = 0
            
            for data, target in test_loader:
                data, target = data.to(device), target.to(device)
                output = model(data)
                loss = criterion(output, target)
                
                test_loss += loss.item()
                _, predicted = output.max(1)
                total += target.size(0)
                correct += predicted.eq(target).sum().item()
            
            # Calculate average loss and accuracy for the epoch
            epoch_loss = test_loss / len(test_loader)
            epoch_acc = 100 * correct / total

            test_losses.append(epoch_loss)
            test_accs.append(epoch_acc)

            print(f"Epoch {epoch+1}/{epochs}, Test Loss: {epoch_loss:.4f}, Test Acc: {epoch_acc:.2f}%")
    
    return test_losses, test_accs


In [ ]:
test_losses, test_accs = test(model, criterion, test_loader, device, epochs=50)

In [ ]:
plot_metrics(train_losses, train_accs, test_losses, test_accs)

## SGD

In [ ]:
model = Deep_Emotion()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=1e-5)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
train_losses, train_accs = train(model, criterion, optimizer, train_loader, device, epochs=20)

In [ ]:
test_losses, test_accs = test(model, criterion, test_loader, device, epochs=20)

In [ ]:
plot_metrics(train_losses, train_accs, test_losses, test_accs)